# 2. SQL

In [1]:
# !pip3 install psycopg2-binary
# https://www.psycopg.org/docs/install.html#quick-install

In [2]:
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [3]:
conn = psycopg2.connect(
    host="158.160.52.106",
    port=5432,
    database="postgres",
    user="student",
    password="JvLda93aA",
)

### Задача 1. Со сколькими креативными агентствами мы работаем? Креативное агентство – это партнер без единой локации, но имеющий патент на хотя бы одну легенду.

In [4]:
query_1 = '''

select count(distinct t.partner_rk) as amount
from
(select p.partner_rk
from 
msu_analytics.Partner p
left join
msu_analytics.Location l
using(partner_rk)
where l.partner_rk is null) t
join
msu_analytics.Legend
using(partner_rk)

        '''

pd.read_sql(query_1, conn)

,amount
0,4


### Задача 2. У какого квеста (укажите его quest_nm) разница доли состоявшихся квестов в декабре 2022 и в январе 2023 наибольшая по модулю? Долей считать количество состоявшихся квестов деленное на количество заявленных. В случае наличия нескольких квестов, подходящих под условие, требуется вывести тот, у которого значение quest_rk больше.

In [5]:
query_2 = '''

select q.quest_rk,
    q.quest_nm,
    proportion
from
(select quest_rk,
    abs(avg(game_flg) filter(where date_part('year', game_dttm) = 2022 and date_part('month', game_dttm) = 12) -
    avg(game_flg) filter(where date_part('year', game_dttm) = 2023 and date_part('month', game_dttm) = 01)) as proportion
from 
msu_analytics.Game g
group by quest_rk) t
left join
msu_analytics.Quest q
using(quest_rk)
where proportion is not null
order by proportion desc, t.quest_rk
limit 1 --можно убрать limit 1 и посмотреть все полученные доли в порядке убывания

        '''

pd.read_sql(query_2, conn)

,quest_rk,quest_nm,proportion
0,9,Реактивный восходящий тренд,1.0


# 3. Python

In [6]:
import numpy as np
import pandas as pd
import math
import random

Для выполнения задания потребуется самому сгенерировать данные с помощью модуля random (все пункты выполняются с помощью нее).

Что необходимо, для создания будущей таблицы:

0. Для генерации данных в random необходимо установить seed = "АДВИ_2023", всего длинна каждого списка $N = 1000$;

1. **index колонка:** создать список из $N$ последовательных целых элементов от 1 до 1000 включительно, далее, с помощью random, перемешать его -> будем использовать дальше как индексы;

2. **groups колонка:** создать список из $N$ элементов, состоящий из значений из списка `["группа_1", "группа_2", "группа_3", "группа_4", "группа_5"]` (равномерное распределение);

3. **uniform колонка:** создать список из N элементов, состоящий из целых значение из равномерного распределения [50, 100];

4. **gauss_1, gauss_2, gauss_3 колонки:** Создать 3 списка из $N$ элементов каждый, заполненные значениями из нормальных распределений с параметрами:
- $\mu = 0, \sigma^2 = 1$ -> третья колонка gauss_1;
- $\mu = 0, \sigma^2 = 144$ -> четвертая колонка gauss_2;
- $\mu = 50, \sigma^2 = 81$ -> пятая колонка gauss_3;
5. Создать pandas DataFrame, где (1) будет индексом, (2)-(4) колонки, названия колонок `["groups", "uniform", "gauss_1", "gauss_2", "gauss_3"]` соответственно, не забывая проверить, что типы колонок должны соответствовать заявленным типам данных;

6. Для колонок `gauss_1` и `gauss_2` заменить значения на None:

- Для `gauss_1`, если индекс у значения делится на 121 без остатка;
- Для `gauss_2`, если дробная часть больше 0.95;
7. Теперь, для пропущенных значений `gauss_1` и `gauss_2`:

- Для `gauss_2` None заменить на среднее по колонке;
- Удалить строчки из таблицы, где `gauss_1 == None`.

### Пункт 0.

In [7]:
random.seed("АДВИ_2023")

### Пункт 1.

In [8]:
index = np.arange(1, 1001)
random.shuffle(index)

### Пункт 2.

In [9]:
groups = random.choices(["группа_1", "группа_2", "группа_3", "группа_4", "группа_5"], k=1000)

### Пункт 3.

In [10]:
uniform = [random.randint(50, 100) for _ in range(1000)]

### Пункт 4.

In [11]:
gauss_1 = [random.gauss(mu=0, sigma=1) for _ in range(1000)]
gauss_2 = [random.gauss(mu=0, sigma=12) for _ in range(1000)]
gauss_3 = [random.gauss(mu=50, sigma=9) for _ in range(1000)]

### Пункт 5.

In [12]:
df = pd.DataFrame(index=index, 
                  columns=["groups", "uniform", "gauss_1", "gauss_2", "gauss_3"]
)

In [13]:
df['groups'] = groups
df['uniform'] = uniform
df['gauss_1'] = gauss_1
df['gauss_2'] = gauss_2
df['gauss_3'] = gauss_3

### Пункт 6.

In [14]:
df.loc[df.index % 121 == 0, 'gauss_1'] = np.nan
df.loc[df.gauss_2.apply(lambda x: x - math.floor(x) > 0.95), 'gauss_2'] = np.nan

### Пункт 7.

In [15]:
df.gauss_2.fillna(df.gauss_2.mean(), inplace=True)
df = df[~df.gauss_1.isna()]

In [16]:
df.head()

,groups,uniform,gauss_1,gauss_2,gauss_3
476,группа_1,95,3.010291,2.156346,57.455851
315,группа_5,77,0.190277,-9.934550,62.694177
490,группа_4,96,-0.282161,9.211071,54.688895
748,группа_3,56,1.127080,-12.391793,30.192353
471,группа_5,52,-1.238270,10.938748,52.682052


### Задача 1. Сколько записей всего получилось?

In [17]:
df.shape[0]

992

### Задача 2. Найти среднее и среднеквадратичное отклонение у `gauss_2`.

In [18]:
round(df.gauss_2.mean(), 2), round(df.gauss_2.std(), 2)

(-0.26, 11.93)

### Задача 3. Какая группа встречается чаще всего и сколько раз?

In [19]:
df.groups.value_counts()

группа_4    222
группа_1    215
группа_2    194
группа_3    184
группа_5    177
Name: groups, dtype: int64

### Задача 4. Найти минимальный индекс из `groups == 'группа_5'`.

In [20]:
df.loc[df.groups == 'группа_5'].index.min()

6

### Задача 5. Сколько элементов в колонке `uniform` больше, чем ее квантиль порядка 0.9?

In [21]:
(df.uniform >= np.quantile(a=df.uniform, q=0.9)).sum()

118

### Задача 6. Найти группу с самой большой медианой в колонке `gauss_3`. Вывести эту группу и медиану.

In [22]:
df.groupby(['groups'], as_index=False).agg({'gauss_3': np.median}) \
    .sort_values('gauss_3', ascending=False).head(1).round(2)

,groups,gauss_3
1,группа_2,49.99


### Задача 7. Вычислить минимумы в колонке `uniform` в разбивке по группам. Среди них выбрать группу с максимальным минимумом (если такой **единственной** нет, то сделать то же самое с колонкой `gauss_1`). Затем у данной группы вычислить минимальное значение в колонке `gauss_2`. В ответе ожидается группа и минимальное значение в колонке `gauss_2` для этой группы.

In [23]:
df.groupby(['groups'], as_index=False).agg({'uniform': min}) # Такого нет

,groups,uniform
0,группа_1,50
1,группа_2,50
2,группа_3,50
3,группа_4,50
4,группа_5,50


In [24]:
df.groupby(['groups'], as_index=False).agg({'gauss_1': min}).sort_values('gauss_1', ascending=False) \
     .head(1)['groups'].values # группа 3

array(['группа_3'], dtype=object)

In [25]:
df.loc[df.groups == 'группа_3'].gauss_2.min().round(2)

-36.19

### Задача 8. Добавить колонку `gauss = (gauss_1 + gauss_2) / gauss_3`. Вычислить у нее квантиль порядка 0.7.

In [26]:
df['gauss'] = (df.gauss_1 + df.gauss_2) / df.gauss_3

In [27]:
np.quantile(a=df.gauss, q=0.7).round(2)

0.11